In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random

In [2]:
def encodestring(var):
    if var is not None:
        return var.encode('utf-8')

In [3]:
def scrapePainting(url):
    r = requests.get(url)
    print r, url
    b = BeautifulSoup(r.text, 'html.parser')
    
    url_id = r.url.split('/')[-1]
    
    artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'})
    if artist is not None:
        artist = b.find('a', {'id':'fine-ART-ProductLabelArtistNameLink'}).text
        
    title = b.find('span', {'id': 'fineArtTitle'})
    if title is not None:
        title = b.find('span', {'id': 'fineArtTitle'}).text
        
    image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
    if image is not None:
        image = re.findall('http://ecx.images-amazon.com/images/I/.*\.jpg', r.text)
        
    price = b.find('span', {'id': 'priceblock_ourprice'})
    if price is not None:
        price = b.find('span', {'id': 'priceblock_ourprice'}).text.replace('$','').replace(',', '')
        if '.' in price:
            price = float(price.replace('.', ''))/100
        else:
            price = float(price)
            
    desc = b.find('div', {'id': 'productDescription_feature_div'})
    if desc is not None:
        desc = b.find('div', {'id': 'productDescription_feature_div'}).getText().replace('\n','')
        
    height = None
    width = None
    size = b.find('span', {'id': 'mnba_buybox_size'})
    if size is not None:
        size = [a.strip() for a in b.find('span', {'id': 'mnba_buybox_size'}).text.split('x')]
        if len(size) == 2:
            height = float(size[0].replace('in.','').replace('in','').strip())
            width = float(size[1].replace('in.','').replace('in','').strip())
            depth = 1.
        elif len(size) == 3:
            height = float(size[0].replace('in.','').replace('in','').strip())
            width = float(size[1].replace('in.','').replace('in','').strip())
            depth = float(size[2].replace('in.','').replace('in','').strip())
    
    size_variations = b.find('div', {'id': 'variation_size_name'})
    if size_variations is not None:
        size = [a.strip() for a in size_variations.find('span', {'class': "a-size-base"}).text.split('x')]
        
        if len(size) == 2:
            height = float(size[0].replace('in.','').replace('in','').strip())
            width = float(size[1].replace('in.','').replace('in','').strip())
            depth = 1.
        elif len(size) == 3:
            height = float(size[0].replace('in.','').replace('in','').strip())
            width = float(size[1].replace('in.','').replace('in','').strip())
            depth = float(size[2].replace('in.','').replace('in','').strip())
            
        price = size_variations.find('span', {'class': "a-size-mini"}).text.replace('$','').replace(',', '')
        if '.' in price:
            price = float(price.replace('.', ''))/100
        else:
            price = float(price)
    

    return {'url_id': encodestring(url_id), 'artist': encodestring(artist), 'title':encodestring(title), 'image': image, 'price': price, 
            'description':encodestring(desc), 'height': height, 'width': width, 'size':size}

In [ ]:
scrapePainting('http://www.amazon.com/Walk-Away-21x21-in/dp/B0170E9ARC')

In [10]:
paintings = []

for page in range(1,219):
    
    page_url = 'http://www.amazon.com/s/ref=lp_6685289011_pg_2?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011&page='+str(page)+'&ie=UTF8&qid=1462681717'
    r = requests.get(page_url)
    r.text

    b = BeautifulSoup(r.text, 'html.parser')

    url_1 = []
    for link in b.findAll('a', {'class':"a-link-normal a-text-normal"}):
        url_1.append(link.get('href'))
    
    url_list = url_1[::2]

    for url in url_list:
        try:
            paintings.append(scrapePainting(url))
        except Exception as error:
            print error
        time.sleep(random.randint(5,20))
    
    if page % 10 == 0:
        df = pd.DataFrame(paintings)
        df.to_csv("output"+str(page)+".csv", index=False)
    elif page == 218:

df = pd.DataFrame(paintings) 
df.to_csv("final_df.csv", index=False)


<Response [200]> http://www.amazon.com/12x18-in-Jack-Androvich-Surfboards/dp/B00EZH6DM8
<Response [503]> http://www.amazon.com/For-sale/dp/B015EOYYEE
<Response [200]> http://www.amazon.com/Steve-Fireman-Beverly-Shores-IN/dp/B00IRTE78C
<Response [503]> http://www.amazon.com/30x40-Simon-Carruthers-Untitled-Pallets/dp/B00LLQ152E
<Response [200]> http://www.amazon.com/Let-Go/dp/B00SVIN0W8
<Response [200]> http://www.amazon.com/Prince/dp/B016VGH73O
<Response [200]> http://www.amazon.com/Fire-Hydrant/dp/B01EKBYJYW
<Response [503]> http://www.amazon.com/Kathy-on-a-Windy-Boat/dp/B00H9MAHUK
<Response [200]> http://www.amazon.com/Ayrton-Senna-F1/dp/B016VGE62E
<Response [503]> http://www.amazon.com/Toy/dp/B00SVITN6A
<Response [503]> http://www.amazon.com/Nude-in-Water/dp/B016VGKAW4
<Response [200]> http://www.amazon.com/Half-Dome-Breaking-Storm/dp/B00NNA11KC
<Response [503]> http://www.amazon.com/fish-splash/dp/B016VGCLNA
<Response [503]> http://www.amazon.com/28x28-Tom-Chambers-Ashly-Deer/dp/B00

In [11]:
df.head()

,0
0,None
1,None
2,None
3,None
4,None


In [41]:
o10 = pd.read_csv('output10.csv')
o210 = pd.read_csv('output210.csv')

In [42]:
o210.describe()

#url_rej = o100[o100.isnull()]
#url_rej.head(50)

,height,price,width
count,4541.000000,4574.000000,4541.000000
mean,18.768168,1539.453824,20.895045
std,10.021583,2644.228700,11.319522
min,0.050000,36.000000,0.000000
25%,12.000000,315.000000,14.000000
50%,16.000000,750.000000,19.000000
75%,22.000000,1800.000000,24.000000
max,107.000000,61000.000000,135.000000


Adding all the rejeected requests

In [ ]:
paintings_rej = final_df.isnull().url_id
paintings = []

for page in range(1,219):
    
    page_url = 'http://www.amazon.com/s/ref=lp_6685289011_pg_3?rh=n%3A4991425011%2Cn%3A%214991426011%2Cn%3A6685269011%2Cn%3A6685289011&page='+str(page)+'&ie=UTF8&qid=1462820989'
    r = requests.get(page_url)
    r.text

    b = BeautifulSoup(r.text, 'html.parser')

    url_1 = []
    for link in b.findAll('a', {'class':"a-link-normal a-text-normal"}):
        url_1.append(link.get('href'))
    
    url_list = url_1[::2]

    for url in url_list:
        if url is in paintings_rej:
            try:
                paintings.append(scrapePainting(url))
            except Exception as error:
                print error
            time.sleep(random.randint(5,20))
    
    if page % 50 == 0:
        df = pd.DataFrame(paintings)
        df.to_csv("rej_output"+str(page)+".csv", index=False)
    elif page == 218:

df = pd.DataFrame(paintings) 
df.to_csv("final_rej_df.csv", index=False)